In [1]:
import numpy as np
import pandas as pd
import psycopg
from sqlalchemy import create_engine
import dotenv
import os
import sqlite3

In [2]:
nba = pd.read_csv('ASA All NBA Raw Data.csv', low_memory=False)

In [6]:
pd.set_option('display.max_columns', 81)
nba.head(3).T

,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0
H_A,A,A,A
Team_Abbrev,WAS,WAS,WAS
...,...,...,...
SG%,36.0,0.0,32.0
SF%,60.0,4.0,67.0
PF%,4.0,85.0,0.0
C%,0.0,11.0,0.0


In [7]:
nba = nba.drop(['Inactives'], axis = 1)

In [8]:
games = nba[['game_id', 'game_date', 'OT', 'season']]
games.head(10)


,game_id,game_date,OT,season
0,202202170BRK,2022-02-17,0,2022
1,202202170BRK,2022-02-17,0,2022
2,202202170BRK,2022-02-17,0,2022
3,202202170BRK,2022-02-17,0,2022
4,202202170BRK,2022-02-17,0,2022
5,202202170BRK,2022-02-17,0,2022
6,202202170BRK,2022-02-17,0,2022
7,202202170BRK,2022-02-17,0,2022
8,202202170BRK,2022-02-17,0,2022
9,202202170BRK,2022-02-17,0,2022


In [15]:
players = nba[['player_id', 'player']].drop_duplicates()


In [10]:
nba = nba.drop(['game_date', 'OT', 'season'], axis = 1)
nba.head(3).T

,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
H_A,A,A,A
Team_Abbrev,WAS,WAS,WAS
Team_Score,117,117,117
Team_pace,94.5,94.5,94.5
...,...,...,...
SG%,36.0,0.0,32.0
SF%,60.0,4.0,67.0
PF%,4.0,85.0,0.0
C%,0.0,11.0,0.0


In [ ]:
nba.columns

Index(['game_id', 'H_A', 'Team_Abbrev', 'Team_Score', 'Team_pace',
       'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct', 'Team_ft_rate',
       'Team_off_rtg', 'Opponent_Abbrev', 'Opponent_Score', 'Opponent_pace',
       'Opponent_efg_pct', 'Opponent_tov_pct', 'Opponent_orb_pct',
       'Opponent_ft_rate', 'Opponent_off_rtg', 'player', 'player_id',
       'starter', 'mp', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft',
       'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'plus_minus', 'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct',
       'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct',
       'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg',
       'def_rtg', 'bpm', 'minutes', 'double_double', 'triple_double', 'DKP',
       'FDP', 'SDP', 'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute',
       'pf_per_minute', 'ts', 'last_60_minutes_per_game_starting',
       'last_60_minutes_per_game_bench'

In [13]:
team_game = nba[['game_id', 'Team_Abbrev', 'H_A', 'Team_Score', 'Team_pace',
       'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct', 'Team_ft_rate',
       'Team_off_rtg', 'Opponent_Abbrev']].drop_duplicates()

In [12]:
player_game = nba.drop(['game_id', 'Team_Abbrev', 'H_A', 'Team_Score', 'Team_pace',
       'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct', 'Team_ft_rate',
       'Team_off_rtg', 'Opponent_Abbrev'], axis=1)

In [14]:
player_game.head(3).T

,0,1,2
Opponent_Score,103,103,103
Opponent_pace,94.5,94.5,94.5
Opponent_efg_pct,0.483,0.483,0.483
Opponent_tov_pct,13.1,13.1,13.1
Opponent_orb_pct,33.3,33.3,33.3
...,...,...,...
SG%,36.0,0.0,32.0
SF%,60.0,4.0,67.0
PF%,4.0,85.0,0.0
C%,0.0,11.0,0.0


In [16]:
team_game.head(3).T

,0,13,26
game_id,202202170BRK,202202170BRK,202202170CHO
Team_Abbrev,WAS,BRK,MIA
H_A,A,H,A
Team_Score,117,103,111
Team_pace,94.5,94.5,88.8
Team_efg_pct,0.627,0.483,0.471
Team_tov_pct,13.5,13.1,11.1
Team_orb_pct,22.9,33.3,26.8
Team_ft_rate,0.157,0.191,0.147
Team_off_rtg,123.8,109.0,103.4


In [17]:
players

,player_id,player
0,kispeco01,Corey Kispert
1,kuzmaky01,Kyle Kuzma
2,caldwke01,Kentavious Caldwell-Pope
3,netora01,Raul Neto
4,bryanth01,Thomas Bryant
...,...,...
109702,frazimi01,Michael Frazier
110441,howarwi01,William Howard
110913,mbahalu01,Luc Mbah a Moute
111399,bowmaky01,Ky Bowman


In [18]:
games

,game_id,game_date,OT,season
0,202202170BRK,2022-02-17,0,2022
1,202202170BRK,2022-02-17,0,2022
2,202202170BRK,2022-02-17,0,2022
3,202202170BRK,2022-02-17,0,2022
4,202202170BRK,2022-02-17,0,2022
...,...,...,...,...
112118,202003070GSW,2020-03-07,0,2020
112119,202003070GSW,2020-03-07,0,2020
112120,202003070GSW,2020-03-07,0,2020
112121,202003070GSW,2020-03-07,0,2020
